In [ ]:
# import & 文件路径
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from Globals import *
from mne.preprocessing import ICA, create_eog_epochs
import csv
%matplotlib qt

In [ ]:
Sub_id = '17'

os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/Sub_0' + Sub_id)
raw = mne.io.read_raw_brainvision('0' + Sub_id + '.vhdr', eog = ['EOG'], preload = True)
raw.set_eeg_reference('average', ch_type = 'eeg')
raw.set_montage('standard_1020')

# 添加坏导
raw.info['bads'].append('AF8')
raw.info['bads'].append('TP10')
#raw.info['bads'].append('P2')
#raw.info['bads'].append('CPz')
#raw.info['bads'].append('AF4')
#raw.info['bads'].append('TP8')
#raw.info['bads'].append('FT8')
#raw.info['bads'].append('C2')
#raw.info['bads'].append('O2')

#raw.info['bads']

# 插值坏导
raw.interpolate_bads()

# 将原始文件中的 annotation 读入 raw_annot 中；
raw_annot = raw.annotations
# make_relax_annotation 函数将 marker 之间间隔超过4.5秒的数据作为休息时间进行标记；
bad_relax_annot = make_relax_annotations(raw)
#将休息时间与原始的 annotation 合并；
raw.set_annotations(raw_annot + bad_relax_annot)
#crop_raw_relax函数对实验末尾的休息时间进行标记；
crop_raw_relax(raw)

raw.filter(0.1,40)
raw_for_ica = raw.copy().filter(1,40)

In [ ]:
os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/ica_fif')
ica = mne.preprocessing.read_ica(Sub_id + '_ica-ica.fif')
ica.plot_components()

In [ ]:
ica.plot_properties(raw_for_ica,[0,3])

In [ ]:
ica.exclude = [0]
ica.apply(raw)

In [ ]:
os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/Sub_0' + Sub_id)
print(os.getcwd())



# 从原始数据的 annotation 中读取 events(包含marker及对应时间点的数组) & event_id(marker名称列表)
events, event_id = mne.events_from_annotations(raw)

# get_corrected_events 函数删除 events 中错误反应以及错误反应对应的刺激marker；
corrected_events = get_corrected_events(events)
# 可视化校正后的 events;
mne.viz.plot_events(corrected_events)
# 对 events 进行切片；
event_1_third , event_2_third = crop_raw_1third_v2(corrected_events)
event_1_two, event_2_two = crop_raw_1two(corrected_events)

# 对切片后的raw数据进行分段；
epochs_1_third = mne.Epochs(raw, events= event_1_third,event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_2_third = mne.Epochs(raw, events= event_2_third, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_1_two = mne.Epochs(raw, events= event_1_two, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_2_two = mne.Epochs(raw, events= event_2_two, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
# 完整数据分段
epochs_all = mne.Epochs(raw, events = corrected_events,event_id = eventdict_1,  reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True )

epochs_1_third.save('1_3_epochs_epo.fif', overwrite = True)
epochs_2_third.save('2_3_epochs_epo.fif', overwrite = True)
epochs_1_two.save('1_2_epochs_epo.fif', overwrite = True)
epochs_2_two.save('2_2_epochs_epo.fif', overwrite = True)
epochs_all.save('epochs_all_epo.fif', overwrite = True)